# Wykrywanie fraudów

## Podstawowe biblioteki i ładowanie danych

In [1]:
import numpy as np
import pandas as pd
import csv
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [125]:
#Wczytujemy dobre dane
df = pd.read_csv("data.csv")

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,127614.0,2.004188,-0.287498,-1.984934,0.342239,0.413188,-0.815346,0.445456,-0.299258,0.562445,...,-0.038661,-0.127194,0.036464,0.504076,0.176423,0.541252,-0.121212,-0.064948,64.99,0
1,139389.0,2.346905,-1.264915,-1.034900,-1.568721,-1.122435,-0.790624,-1.011732,-0.268243,-1.229191,...,-0.196288,0.015015,0.197023,-0.432445,-0.119139,-0.131627,0.015176,-0.058104,1.00,0
2,68482.0,-3.089671,2.464689,0.042745,1.670674,-2.129076,0.600262,-1.627286,2.452801,0.319063,...,-0.223384,-0.441265,0.250283,0.091483,-0.038373,-0.410096,0.088944,0.026973,30.70,0
3,123677.0,1.792358,-0.384458,-2.170063,0.096243,0.291162,-1.510182,0.921037,-0.525263,-0.088484,...,0.297749,0.633531,-0.176001,0.146116,0.342753,0.198887,-0.117734,-0.061706,154.09,0
4,121222.0,2.188008,-0.400212,-1.627908,-0.225692,-0.133549,-1.273351,0.226144,-0.553975,-0.908684,...,-0.338599,-0.220436,0.112804,0.049313,0.157236,0.606729,-0.063450,-0.071105,10.00,0


In [4]:
#feature selection based on EDA on other file
df_thick = df.drop(columns=['V5', 'V6', 'V8', 'V13', 'V15', 'V15', 'V20','V21','V22','V23','V24','V25', 'V26','V27','V28'])

In [5]:
X = df.drop(columns = 'Class')
y = df['Class']
x_t = df_thick.drop(columns= 'Class')

## Model XGBoost

In [6]:
#Podział na zbiór uczacy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1279)
X_t_train, X_t_test, y_train, y_test = train_test_split(x_t, y, test_size=0.25, random_state=1279)

In [7]:
#Do XGBoosta, wymaga Dmatrix do treningu
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_train_t = xgb.DMatrix(X_t_train, label = y_train)
xgb_test = xgb.DMatrix(X_test, label = y_test)
xgb_test_t = xgb.DMatrix(X_t_test, label =y_test)

/home/arkadiusz/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [98]:
#Ustawiamy parametry losowo
params = {
    'eta': 0.3,
    'max_depth': 7,
    'objectice': 'binary:logistic',
    'eval_metric': 'aucpr',
    'scale_pos_weight': 10,
}
evallist = [(xgb_test, 'eval'), (xgb_train, 'train')]

In [101]:
#trenujemy przykładowy model
num_round = 15
model = xgb.train(params, xgb_train, num_round, evallist)

[0]	eval-aucpr:0.732611	train-aucpr:0.864914
[1]	eval-aucpr:0.758561	train-aucpr:0.874183
[2]	eval-aucpr:0.76063	train-aucpr:0.893356
[3]	eval-aucpr:0.784341	train-aucpr:0.909237
[4]	eval-aucpr:0.784799	train-aucpr:0.924271
[5]	eval-aucpr:0.7852	train-aucpr:0.930419
[6]	eval-aucpr:0.785627	train-aucpr:0.938798
[7]	eval-aucpr:0.790592	train-aucpr:0.940695
[8]	eval-aucpr:0.789856	train-aucpr:0.943571
[9]	eval-aucpr:0.790892	train-aucpr:0.950527
[10]	eval-aucpr:0.800599	train-aucpr:0.955976
[11]	eval-aucpr:0.799327	train-aucpr:0.96033
[12]	eval-aucpr:0.795983	train-aucpr:0.967886
[13]	eval-aucpr:0.792223	train-aucpr:0.977094
[14]	eval-aucpr:0.791726	train-aucpr:0.978801


In [117]:
#przeszukujemy przestrzeń parametrów w celu znalezienie najlepszej kombinacji
param_to_tune = {
    "learning_rate":[0.1, 0.2, 0.3],
    "max_depth": np.arange(5,10),
    "scale_pos_weight": [1, 10, 20],
    'objectice': ['binary:logistic'],
    'n_estimators': [10,20, 30, 40],
    'subsample': [0.7, 0.8, 0.9, 1]
        }
modelGD = xgb.XGBClassifier()
grid = GridSearchCV(modelGD, param_to_tune, cv = 7, scoring= 'average_precision')
grid.fit(X_train, y_train, eval_metric = 'aucpr')


GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_l...
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_depth': array([5, 6, 7, 8, 9]),
                         'n_estimators': [10, 20, 3

In [118]:
#Podglądamy rezultat
best = grid.best_estimator_
print(best)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.3, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=40, n_jobs=1,
              nthread=None, objectice='binary:logistic',
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.7, verbosity=1)


In [119]:
#rezultat może być zanliżony, przez to że nie zwraca prawdopodobieństwa domyślnie
best_score = grid.best_score_
print(best_score)

0.855379709943619


In [124]:
#zapisujemy
best.save_model("best_model_full_data")

In [120]:
#predykujemy na testowym
y_pred = best.predict_proba(X_test)[:, 1]

In [121]:
average_precision_score(y_test, y_pred)

0.8162356090559539

In [122]:
#porównanie: model bez zoptymalizowanych parametrów
model_pred = model.predict(xgb_test)

In [123]:
average_precision_score(y_test, model_pred)

0.7901594880145533

In [126]:
param_to_tune = {
    "learning_rate":[0.1, 0.2, 0.3],
    "max_depth": np.arange(5,10),
    "scale_pos_weight": [1, 10, 20],
    'objectice': ['binary:logistic'],
    'n_estimators': [10,20, 30, 40],
    'subsample': [0.7, 0.8, 0.9, 1]
        }
modelGD = xgb.XGBClassifier()
grid = GridSearchCV(modelGD, param_to_tune, cv = 7, scoring= 'average_precision')
grid.fit(X_t_train, y_train, eval_metric = 'aucpr')


GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_l...
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_depth': array([5, 6, 7, 8, 9]),
                         'n_estimators': [10, 20, 3

In [127]:
other_best = grid.best_estimator_
print(other_best)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.3, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=40, n_jobs=1,
              nthread=None, objectice='binary:logistic',
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=20, seed=None, silent=None,
              subsample=0.7, verbosity=1)


In [128]:
y_other_pred = other_best.predict_proba(X_t_test)[:, 1]
average_precision_score(y_test, y_other_pred)

0.8116030821350344

In [131]:
other_best.save_model("bm_filter_data")